In [2]:
dataset_train = "dataset\\ingatlan.com_training_final.csv"
dataset_val = "dataset\\ingatlan.com_testing_final.csv"
dataset = "dataset\\ingatlan.com_full_final.csv"
RANDOM_STATE = 39612
import numpy as np
import matplotlib as plt
import pandas as pd
import re
import roman
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

In [3]:
mldf = pd.read_csv('finalized_for_train.csv')
mldf

,city,postcode,property_subtype,property_condition_type,property_floor,building_floor_count,view_type,orientation,garden_access,heating_type,elevator_type,room_cnt,small_room_cnt,created_at,property_area,balcony_area,price_created_at,ad_view_cnt,active_days
0,1,1015.0,1,2,4,-1,0.0,1,0,3,1,1.0,0.0,354,40.0,4.0,22.9,128.0,16.0
1,1,1012.0,1,3,3,2,2.0,2,0,2,1,1.0,1.0,435,70.0,0.0,33.0,63.0,34.0
2,1,1016.0,1,4,3,-1,0.0,2,0,1,0,1.0,1.0,302,40.0,1.0,21.5,85.0,72.0
3,1,1016.0,1,2,0,-1,0.0,0,0,2,0,1.0,0.0,129,35.0,0.0,13.0,53.0,13.0
4,1,1015.0,1,3,1,1,1.0,0,0,2,1,1.0,2.0,302,55.0,0.0,33.5,153.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248240,13,1131.0,1,3,1,0,3.0,2,1,1,0,2.0,0.0,564,50.0,0.0,15.0,268.0,3.0
248241,12,1121.0,1,1,2,-1,0.0,0,0,1,0,2.0,0.0,564,65.0,6.0,23.9,187.0,4.0
248242,11,1111.0,1,4,3,-1,0.0,2,0,0,1,1.0,1.0,565,55.0,3.0,33.5,48.0,1.0
248243,8,1081.0,1,2,3,4,0.0,2,0,0,1,2.0,0.0,566,55.0,8.0,18.0,115.0,3.0


In [4]:
def train_eval_linreg(df,target_cols):
    X = df[target_cols]
    y = df['price_created_at']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

    model = LinearRegression()
    model.fit(X_train, y_train)


    y_pred = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    print(f'LR Mean Squared Error: {mse}')
    mape = mean_absolute_percentage_error(y_test, y_pred)*100
    print(f'LR Mean Absolute Percentage Error: {mape}')

def train_eval_gbm_gridsearch(df,target_cols):
    X = df[target_cols]
    y = df['price_created_at']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

    param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 4, 5]
    }

    gbm = GradientBoostingRegressor(random_state=RANDOM_STATE)
    grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_percentage_error', n_jobs=-1)

    # Fit GridSearchCV
    grid_search.fit(X_train, y_train)
    best_gbm = grid_search.best_estimator_
    best_gbm.fit(X_train, y_train)

    y_pred = best_gbm.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    print(f"GBM Mean Squared Error: {mse}")
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    print(f"GBM Mean Absolute Percentage Error: {mape}")
    print(f"Best GBM params {best_gbm.get_params}")

def train_eval_gbm(df,target_cols):
    X = df[target_cols]
    y = df['price_created_at']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

    gbm = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=5, random_state=RANDOM_STATE)

    gbm.fit(X_train, y_train)
    y_pred = gbm.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"GBM Mean Squared Error: {mse}")
    mape = mean_absolute_percentage_error(y_test, y_pred)*100
    print(f"GBM Mean Absolute Percentage Error: {mape}")
    #return(mse,mape)


#target_cols = ['property_area', 'room_cnt', 'small_room_cnt', 'balcony_area', 'elevator_type',
#               'city','property_subtype','garden_access','heating_type','orientation',
#               'property_floor', 'building_floor_count', 'property_condition_type',
#               'created_at']


---

> Tanítási start

In [5]:
mldf['meroszam'] = (mldf['ad_view_cnt'] / mldf['active_days'])*100

In [6]:
target_cols = mldf.columns
target_cols = target_cols.drop('meroszam')
#target_cols = target_cols.drop('active_days')
#target_cols = target_cols.drop('ad_view_cnt')
target_cols = target_cols.drop('price_created_at')

train_eval_linreg(mldf,target_cols)
train_eval_gbm_gridsearch(mldf,target_cols)
#train_eval_gbm(mldf,target_cols)

LR Mean Squared Error: 32.78822098852316
LR Mean Absolute Percentage Error: 24.087435831977263
GBM Mean Squared Error: 11.191825981056413
GBM Mean Absolute Percentage Error: 11.70590575584258
Best GBM params <bound method BaseEstimator.get_params of GradientBoostingRegressor(max_depth=5, n_estimators=300, random_state=39612)>
